In [1]:
import pandas as pd
import pickle

from tqdm import tqdm

tqdm.pandas()

import torch
from torch.utils.data import Dataset


import torch, torch.nn as nn
from transformers import Qwen2_5OmniForConditionalGeneration, Qwen2_5OmniProcessor
from peft import get_peft_model, LoraConfig, TaskType


from torch.nn.utils.rnn import pad_sequence
from transformers import BatchEncoding

In [2]:
path_to_dataset = "/mnt/core-llm/unisrec/data/raw/user_history_users_8_000_000_nmid_1_000_000_raw_2024_05_30"
train_df = pd.read_parquet(path_to_dataset)

In [3]:
# Фиксируем random seed для воспроизводимости
random_seed = 42

# Выделяем 10% случайных индексов для валидации
val_df = train_df.sample(frac=0.1, random_state=random_seed)

# Убираем эти строки из train_df
train_df = train_df.drop(val_df.index)

# Из оставшегося train_df выделяем 10% для теста
test_df = train_df.sample(frac=0.1, random_state=random_seed)

# Убираем эти строки из train_df
train_df = train_df.drop(test_df.index)

print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")


Train size: 6458215
Validation size: 797310
Test size: 717580


In [4]:
with open("/mnt/core-llm/unisrec/data/downstream_text_cv_50M_users_1kk_nm_ids/cache/filtered_item2index.pkl", "rb") as f:
    filtered_item2index = pickle.load(f)

# Создаём обратный словарь item -> index
filtered_index2item = {idx: item for item, idx in filtered_item2index.items()}


In [5]:
def prepare_train_examples(df, filtered_item2index):

    def process_nm_ids(nm_ids):
        return [
            filtered_item2index[nm_id]
            for nm_id in nm_ids
            if nm_id in filtered_item2index
        ]
    
    df["nm_ids"] = df["nm_ids"].progress_apply(process_nm_ids)

    # Фильтруем строки с <2 интеракциями
    df = df[df["nm_ids"].progress_apply(len) > 1]

    examples = []

    for row in tqdm(df.itertuples(index=False), total=len(df), desc="Building examples"):
        user_id = row.user_id
        ids = row.nm_ids
        titles = row.titles

        for i in range(1, len(ids)):
            examples.append({
                "text": f"user {user_id}, history purchase: " + ", ".join(titles[:i]),
                "ids": ids[:i],
                "target": ids[i]
            })

    return pd.DataFrame(examples)


In [6]:
class RecommendationDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        conversation = [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": (
                            "You are a recommendation system assistant designed to provide personalized product suggestions "
                            "based on user purchase history and preferences. You can process both textual information and "
                            "item IDs representing products the user interacted with. Respond clearly and helpfully."
                        )
                    }
                ],
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": row['text']},
                    # {"type": "id", "id": row['ids']}
                ]
            },
        ]
        
        target = row['target']
        ids = row['ids']
        return conversation, ids, target


In [7]:
train_examples_df = prepare_train_examples(train_df, filtered_item2index)
val_examples_df = prepare_train_examples(val_df, filtered_item2index)  # Аналогично, для валидации

train_ds = RecommendationDataset(train_examples_df)
val_ds = RecommendationDataset(val_examples_df)

Building examples: 100%|██████████| 647992/647992 [00:24<00:00, 26141.25it/s]


In [8]:
from transformers import Qwen2_5OmniProcessor, Qwen2_5OmniThinkerForConditionalGeneration

# Используем Thinker, у него forward реализован
model = Qwen2_5OmniThinkerForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-Omni-7B", torch_dtype=torch.float16, device_map="auto"
)
processor = Qwen2_5OmniProcessor.from_pretrained("Qwen/Qwen2.5-Omni-7B")

text = "user 232313, history purchase: белые джинсы, розовая помада"
inputs = processor.tokenizer(text, return_tensors="pt").to(model.device)

outputs = model(**inputs, output_hidden_states=True, return_dict=True)
# print(outputs.hidden_states)
# print(len(outputs.hidden_states))
last_hidden_state = outputs.hidden_states[-1]
print(last_hidden_state.shape)


Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

torch.Size([1, 25, 3584])


In [10]:
import torch
import torch.nn as nn
from transformers import Qwen2_5OmniForConditionalGeneration, Qwen2_5OmniProcessor, Qwen2_5OmniThinkerForConditionalGeneration
from peft import get_peft_model, LoraConfig, TaskType
from torch.nn.utils.rnn import pad_sequence
 

class QwenTextEncoder(nn.Module):
    def __init__(self, ckpt="Qwen/Qwen2.5-Omni-7B", reduced_dim=1024, device='cpu'):
        super().__init__()
        self.device = device
        self.model = Qwen2_5OmniThinkerForConditionalGeneration.from_pretrained(
            ckpt, torch_dtype=torch.float16, device_map={"" : str(device)}
        )
        # device_map="auto"
        self.processor = Qwen2_5OmniProcessor.from_pretrained(ckpt)
        self.hidden_size = self.model.config.text_config.hidden_size
        self.reduced_dim = reduced_dim
        
        self.pool_proj = nn.Linear(self.hidden_size, self.reduced_dim).to(torch.float16)

        for name, param in self.model.named_parameters():
            param.requires_grad = False

    def forward(self, text):
        inputs = self.processor.tokenizer(
            text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(self.device)

        outputs = self.model(**inputs, output_hidden_states=True, return_dict=True)
        # print(outputs.hidden_states)
        # print(len(outputs.hidden_states))
        last_hidden_state = outputs.hidden_states[-1]
        # print(f"last_hidden_state = {last_hidden_state.shape}")
        pooled = last_hidden_state.mean(dim=1)  # mean pooling по seq_len -> (B, D)
        # print(f"pooled.shape = {pooled.shape}")
        projected = self.pool_proj(pooled)
        # print(f"projected.shape = {projected.shape}")
        return projected


class FusionHead(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.dense1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        return x

class QwenOmniWithID(nn.Module):
    def __init__(self, ckpt, id_vocab_size=1_000_000, id_dim=512, fusion_dim=1024, reduced_dim=1024, device='cpu'):
        super().__init__()
        self.text_encoder = QwenTextEncoder(ckpt, reduced_dim, device)
        
        self.id_emb = nn.Embedding(id_vocab_size, id_dim)
        self.id_proj = nn.Linear(id_dim, self.text_encoder.reduced_dim)
        
        # Fusion head для предсказания следующего id
        self.fusion_head = FusionHead(
            input_dim=self.text_encoder.reduced_dim * 2,
            hidden_dim=fusion_dim,
            output_dim=id_vocab_size
        )


    def forward(self, text=None, id_ids=None, labels=None):
        # Текстовый эмбеддинг
        text_emb = self.text_encoder(text)  # (B, hidden_size)
        
        # ID эмбеддинг
        if id_ids is not None:
            ids_tensor = pad_sequence(
                [torch.tensor(ids, dtype=torch.long) for ids in id_ids],
                batch_first=True,
                padding_value=0
            ).to(text_emb.device)
            id_embeds = self.id_emb(ids_tensor).mean(dim=1)  # (B, id_dim)
            id_proj = self.id_proj(id_embeds)  # (B, hidden_size)
        else:
            id_proj = torch.zeros_like(text_emb)

        print(f"id_proj = {id_proj.shape}")
        # Fusion (конкатенация)
        fused = torch.cat([text_emb, id_proj], dim=1)  # (B, hidden_size*2)
        print(f"fused shape = {fused.shape}")
        logits = self.fusion_head(fused)  # (B, id_vocab_size)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
        
        return {"loss": loss, "logits": logits}


In [11]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

def collate_fn(batch):
    conversations, id_ids, targets = zip(*batch)
    
    texts = [c[1]['content'][0]['text'] for c in conversations]  # user text
    id_seqs = list(id_ids)
    labels = torch.tensor(targets, dtype=torch.long)

    return {
        "text": texts,
        "id_ids": id_seqs,
        "labels": labels
    }

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_ds, batch_size=8, shuffle=False, collate_fn=collate_fn)


In [12]:
from transformers import get_scheduler
from peft import prepare_model_for_kbit_training

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = QwenOmniWithID(ckpt="Qwen/Qwen2.5-Omni-7B", id_vocab_size=len(filtered_item2index), reduced_dim=1024, device=device).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

num_epochs = 3
num_training_steps = len(train_loader) * num_epochs
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps
)


Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
from tqdm import tqdm

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} [train]"):
        batch = {k: v for k, v in batch.items()}
        batch["text"] = batch["text"]  # already list[str]
        batch["id_ids"] = batch["id_ids"]
        batch["labels"] = batch["labels"].to(device)

        outputs = model(
            text=batch["text"],
            id_ids=batch["id_ids"],
            labels=batch["labels"]
        )

        loss = outputs["loss"]
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} Train Loss: {avg_train_loss:.4f}")

    ### Eval ###
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Epoch {epoch+1} [val]"):
            batch["labels"] = batch["labels"].to(device)
            outputs = model(
                text=batch["text"],
                id_ids=batch["id_ids"],
                labels=batch["labels"]
            )
            val_loss += outputs["loss"].item()

            logits = outputs["logits"]
            preds = logits.argmax(dim=-1)
            correct += (preds == batch["labels"]).sum().item()
            total += len(batch["labels"])

    acc = correct / total
    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1} Val Loss: {avg_val_loss:.4f}, Accuracy: {acc:.4f}")


Epoch 1 [train]:   0%|          | 0/4699119 [00:00<?, ?it/s]

last_hidden_state = torch.Size([8, 512, 3584])
pooled.shape = torch.Size([8, 3584])
projected.shape = torch.Size([8, 1024])


Epoch 1 [train]:   0%|          | 1/4699119 [00:02<3502:39:48,  2.68s/it]

id_proj = torch.Size([8, 1024])
fused shape = torch.Size([8, 2048])
last_hidden_state = torch.Size([8, 512, 3584])
pooled.shape = torch.Size([8, 3584])
projected.shape = torch.Size([8, 1024])


Epoch 1 [train]:   0%|          | 2/4699119 [00:03<1799:54:45,  1.38s/it]

...

id_proj = torch.Size([8, 1024])
fused shape = torch.Size([8, 2048])
last_hidden_state = torch.Size([8, 129, 3584])
pooled.shape = torch.Size([8, 3584])
projected.shape = torch.Size([8, 1024])
id_proj = torch.Size([8, 1024])
fused shape = torch.Size([8, 2048])


Epoch 1 [train]:   0%|          | 6899/4699119 [42:49<407:23:44,  3.20it/s]

last_hidden_state = torch.Size([8, 135, 3584])
pooled.shape = torch.Size([8, 3584])
projected.shape = torch.Size([8, 1024])
id_proj = torch.Size([8, 1024])
fused shape = torch.Size([8, 2048])
last_hidden_state = torch.Size([8, 177, 3584])
pooled.shape = torch.Size([8, 3584])
projected.shape = torch.Size([8, 1024])
id_proj = torch.Size([8, 1024])
fused shape = torch.Size([8, 2048])


Epoch 1 [train]:   0%|          | 6899/4699119 [42:50<485:34:43,  2.68it/s]


KeyboardInterrupt: 